# Рекуррентные нейронные сети

# Векторизация слов

In [1]:
import codecs
import numpy as np
import gensim
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence
import multiprocessing
from sklearn.feature_extraction.text import CountVectorizerx
import re
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
import hashlib
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kuzga\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
def filterText(input_path, output_path, encoding='utf-8'):
    inp = open(input_path, "r", encoding=encoding)
    otp = open(output_path, "w", encoding=encoding)

    content = inp.read().lower()
    # filtered = ''.join(c for c in content if c.isalpha() or c.isspace() or c == '.')
    # filtered = ''.join(c for c in content if c.isalpha() or c.isspace())
    filtered = ''.join(c for c in content if (c >= 'а' and c <= 'я') or c.isspace())
    otp.write(filtered)

filterText("./data/text.txt", "./data/filtered.txt")

In [3]:
params = {
    'vector_size': 100,          # Размерность векторов слов
    'window': 5,          # Максимальное расстояние между текущим и прогнозируемым словом в предложении
    'min_count': 1,       # Минимальная частота слова для учета в словаре
    'workers': multiprocessing.cpu_count(),  # Количество ядер CPU для обучения модели параллельно
}

# Обучение модели Word2Vec
model = Word2Vec(LineSentence('./data/filtered.txt'), **params)

word_vectors = model.wv
similar_words = word_vectors.most_similar('он', topn=5)
similar_words

[('его', 0.9997251033782959),
 ('ее', 0.9997027516365051),
 ('и', 0.9996612668037415),
 ('раскольников', 0.9996235370635986),
 ('опять', 0.9995976090431213)]

In [4]:
print('Похожие слова:')
for word in ["он", "старуха", "топор", "кровь", "убийство", "деньги", "смерть", "смысл"]:
    most_similar = ', '.join('%s (%.2f)' % (similar, dist) for similar, dist in model.wv.most_similar(word)[:8])
    print('  %s -> %s' % (word, most_similar))

Похожие слова:
  он -> его (1.00), ее (1.00), и (1.00), раскольников (1.00), опять (1.00), ему (1.00), лицо (1.00), голову (1.00)
  старуха -> когда (1.00), чтото (1.00), долго (1.00), особенно (1.00), между (1.00), одно (1.00), ним (1.00), свидригайлов (1.00)
  топор -> даже (1.00), но (1.00), себе (1.00), более (1.00), когда (1.00), ей (1.00), наконец (1.00), особенно (1.00)
  кровь -> без (1.00), им (1.00), всего (1.00), здесь (1.00), где (1.00), тоже (1.00), их (1.00), после (1.00)
  убийство -> знаешь (0.98), тебя (0.98), куда (0.98), меня (0.98), романыч (0.98), история (0.98), хочешь (0.98), зачем (0.98)
  деньги -> всего (1.00), тоже (1.00), действительно (1.00), эти (1.00), без (1.00), совершенно (1.00), много (1.00), были (1.00)
  смерть -> весьма (0.99), чемнибудь (0.99), должен (0.99), подумал (0.99), скоро (0.99), могла (0.99), ответил (0.99), улицы (0.99)
  смысл -> мог (0.99), ясно (0.99), имеет (0.99), сестру (0.99), сонечка (0.99), было (0.98), только (0.98), известно 

In [5]:
# Получение вектора
model.wv.get_vector('топор')

array([-0.08965661,  0.1626348 ,  0.05584126,  0.08105635,  0.09044381,
       -0.28404063,  0.05868414,  0.43906528, -0.20381114, -0.05343307,
       -0.04014989, -0.27966794, -0.01161712,  0.20411296, -0.02238424,
       -0.17301746,  0.26387173, -0.12300192, -0.06516097, -0.47820947,
        0.06805194,  0.1534325 ,  0.2932184 , -0.14247939,  0.03667761,
       -0.03962792, -0.3030615 , -0.08629414, -0.1431514 ,  0.02990577,
        0.17477962, -0.08782525,  0.14064163, -0.1550522 , -0.10518517,
        0.08545351,  0.02628002, -0.25578168, -0.15659773, -0.32207623,
       -0.00234981, -0.20696104, -0.16673952,  0.12444065,  0.13367817,
       -0.07150555, -0.13726112, -0.10823711,  0.14628184,  0.09771533,
        0.09277186, -0.16960654, -0.12088121, -0.03999675, -0.16595332,
       -0.08881006,  0.12778117, -0.09791247, -0.19467181,  0.07286628,
        0.06190931, -0.00967498,  0.08910484,  0.00071391, -0.36798918,
        0.25386113,  0.02657666,  0.24528365, -0.420159  ,  0.28

In [6]:
# def get_sentences(text, sentence_len):
#     # sentences = [sent for doc in text for sent in doc.split('.')]
#     # sentences = [sent for doc in text for sent in doc.split('.')]
#     sentences = re.split(r'[.!?]', text)
#     sentences = [
#         [word for word in sent.lower().split()[:sentence_len] if word]
#                  for sent in sentences if len(sent.lower().split()[:sentence_len]) == sentence_len
#     ]
#     return sentences
def get_sentences(text, exact_sentence_len):
    # Разделение текста на предложения
    sentences = sent_tokenize(text)

    # Создание предложений точно определенной длины в символах с учетом слов
    result_sentences = []
    current_sentence = ""
    for sentence in sentences:
        words = word_tokenize(sentence)
        for word in words:
            if len(current_sentence) + len(word) + 1 <= exact_sentence_len:
                current_sentence += word + " "
            else:
                if len(current_sentence) == exact_sentence_len:
                    result_sentences.append(current_sentence.strip())
                current_sentence = word + " "

    # Добавление последнего предложения, если оно не пусто
    if current_sentence and len(current_sentence) == exact_sentence_len:
        result_sentences.append(current_sentence.strip())

    return result_sentences

a = open("./data/filtered.txt", "r", encoding="utf-8")
text = a.read().lower()
sentences = get_sentences(text, 10)
a.close()
sentences

['время под',
 'жильцов в',
 'встречи с',
 'под самою',
 'каморку с',
 'прислугой',
 'лестницей',
 'отдельной',
 'выходе на',
 'улицу ему',
 'надо было',
 'проходить',
 'трусливое',
 'хозяйке и',
 'труслив и',
 'состоянии',
 'углубился',
 'уединился',
 'не только',
 'встречи с',
 'бедностью',
 'положение',
 'перестало',
 'последнее',
 'насущными',
 'своими он',
 'хозяйки в',
 'что бы та',
 'замышляла',
 'обыденную',
 'дребедень',
 'о платеже',
 'лгать нет',
 'какнибудь',
 'кошкой по',
 'раз страх',
 'встречи с',
 'по выходе',
 'дело хочу',
 'и в то же',
 'гм да все',
 'любопытно',
 'чего люди',
 'слова они',
 'впрочем я',
 'ничего не',
 'делаю что',
 'впрочем и',
 'ничего не',
 'делаю это',
 'последний',
 'горохе ну',
 'иду разве',
 'разве это',
 'совсем не',
 'к тому же',
 'пыль и та',
 'особенная',
 'известная',
 'это разом',
 'неприятно',
 'которых в',
 'особенное',
 'множество',
 'поминутно',
 'на буднее',
 'довершили',
 'омерзения',
 'мелькнуло',
 'кстати он',
 'строен но',
 'забы

In [8]:
import re
import numpy as np


def one_hot_encoding(sentence, alphabet):
    # print(sentence)
    char_to_index = {char: i for i, char in enumerate(alphabet)}
    num_unique_chars = len(alphabet)
    one_hot_matrix = np.zeros((len(sentence), num_unique_chars))

    for i, char in enumerate(sentence):
        one_hot_matrix[i, char_to_index[char]] = 1

    return one_hot_matrix

# # Создаем строку из всех символов русского алфавита
russian_alphabet = "абвгдеёжзийклмнопрстуфхцчшщъыьэюя "

# # Применяем one-hot encoding к каждому символу в каждом предложении
encoding_size = len(russian_alphabet)
encoded_sentences = [one_hot_encoding(sentence, russian_alphabet) for sentence in sentences]

encoded_array = np.array(encoded_sentences)

# # Выводим результат
print(encoded_array.shape)
np.set_printoptions(precision=0, suppress=True, threshold=np.inf)
print(sentences[:1])
print(encoded_array[:1])

(22001, 9, 34)
['время под']
[[[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 

In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt

encoded_array_torch = torch.tensor(encoded_array, dtype=torch.float32)

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.softmax = nn.Softmax(dim=-1)  # Добавляем softmax

    def forward(self, x):
        out, _ = self.rnn(x)
        out = self.fc(out)
        out = self.softmax(out)  # Применяем softmax
        return out


input_size = encoded_array.shape[2]
hidden_size = 128
output_size = encoded_array.shape[2]
model = RNN(input_size, hidden_size, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 500
for epoch in range(num_epochs):
    outputs = model(encoded_array_torch)
    loss = criterion(outputs.view(-1, output_size), encoded_array_torch.view(-1, output_size))

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

torch.save(model.state_dict(), './data/rnn13.pth')
print("weight were saved")

Epoch [1/500], Loss: 3.5268
Epoch [2/500], Loss: 3.5261
Epoch [3/500], Loss: 3.5254
Epoch [4/500], Loss: 3.5246
Epoch [5/500], Loss: 3.5238
Epoch [6/500], Loss: 3.5229
Epoch [7/500], Loss: 3.5219
Epoch [8/500], Loss: 3.5207
Epoch [9/500], Loss: 3.5192
Epoch [10/500], Loss: 3.5174
Epoch [11/500], Loss: 3.5151
Epoch [12/500], Loss: 3.5123
Epoch [13/500], Loss: 3.5091
Epoch [14/500], Loss: 3.5058
Epoch [15/500], Loss: 3.5025
Epoch [16/500], Loss: 3.4994
Epoch [17/500], Loss: 3.4965
Epoch [18/500], Loss: 3.4939
Epoch [19/500], Loss: 3.4914
Epoch [20/500], Loss: 3.4891
Epoch [21/500], Loss: 3.4869
Epoch [22/500], Loss: 3.4848
Epoch [23/500], Loss: 3.4828
Epoch [24/500], Loss: 3.4809
Epoch [25/500], Loss: 3.4791
Epoch [26/500], Loss: 3.4773
Epoch [27/500], Loss: 3.4757
Epoch [28/500], Loss: 3.4741
Epoch [29/500], Loss: 3.4726


KeyboardInterrupt: 

In [18]:
input_size = encoded_array.shape[2]
hidden_size = 128
output_size = encoded_array.shape[2]
model = RNN(input_size, hidden_size, output_size)
model.load_state_dict(torch.load('./data/rnn13.pth'))

def generate_sequence(model, start_sequence, length):
    generated_sequence = start_sequence.copy()

    for _ in range(length):
        input_sequence = torch.tensor([one_hot_encoding(generated_sequence, russian_alphabet)], dtype=torch.float32)
        output_probs = model(input_sequence)
        predicted_probs = output_probs[0][-1].detach().numpy()
        # predicted_probs = np.maximum(predicted_probs, 0.0)
        predicted_probs /= np.sum(predicted_probs)
        next_char_index = np.random.choice(len(predicted_probs), p=predicted_probs)
        next_char = russian_alphabet[next_char_index]
        generated_sequence.append(next_char)
    return generated_sequence


start_sequence = "он"
generated_sequence = generate_sequence(model, list(start_sequence), length=50)
print("Generated Sequence:", "".join(generated_sequence))

Generated Sequence: оннннооммммьмааа   тттттттнннннноееееееееееееееее   


# Задание
1. Подготовьте данные для word2vec по одной из недавно прочитанных книг, удалив все символы, кроме букв и пробелов и обучите модель. Посмотрите результат.
2. Для обучения на нефтяных скважин добавьте во входные данные информацию со столбцов Gas, Water (т.е. размер x_data будет (440, 12, 3)) и обучите новую модель. Выход содержит Liquid, Gas и Water (для дальнейшего предсказания). Графики с результатами только для Liquid.
3. Из этого же текста (п.1) возьмите небольшой фрагмент, разбейте на предложения с одинаковым числом символов. Каждый символ предложения закодируйте с помощью one hot encoding. В итоге у вас должен получиться массив размера (n_sentences, sentence_len, encoding_size).
4. На полученных в п.3 задании обучение модель RNN для предсказания следующего символа. Посмотрите результат при последовательной генерации.
5. \* (не обязательное) На полученных в п.1 задании обучение модель RNN для предсказания следующего слова. Посмотрите результат при последовательной генерации.

In [22]:
import pandas as pd

df = pd.read_csv('./data/production.csv')
df.head()

liquid = df.groupby('API')['Liquid'].apply(lambda df_: df_.reset_index(drop=True))
liquid.head()

API             
5005072170100  0     9783
               1    24206
               2    20449
               3     6820
               4     7349
Name: Liquid, dtype: int64